In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

labels = train["Survived"]
train.drop(columns = ["Survived"], inplace = True)

traintest = pd.concat([train, test])

In [0]:
traintest.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
traintest.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [0]:
traintest.drop(columns = ["PassengerId", "Cabin", "Fare", "Ticket"], inplace = True)

In [0]:
traintest.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         263
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [0]:
Sex = pd.get_dummies(traintest["Sex"])
temp = Sex["male"]
traintest["Sex"] = temp

In [0]:
traintest.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,C
2,3,"Heikkinen, Miss. Laina",0,26.0,0,0,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,S
4,3,"Allen, Mr. William Henry",1,35.0,0,0,S


In [0]:
temp = traintest["Embarked"]
temp = pd.factorize(temp, na_sentinel = -1)
traintest["Embarked"] = temp[0]

In [0]:
traintest.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,1
2,3,"Heikkinen, Miss. Laina",0,26.0,0,0,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,0
4,3,"Allen, Mr. William Henry",1,35.0,0,0,0


In [0]:
traintest.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         263
SibSp         0
Parch         0
Embarked      0
dtype: int64

In [0]:
traintest["Name"].unique()

array(['Braund, Mr. Owen Harris',
       'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
       'Heikkinen, Miss. Laina', ..., 'Saether, Mr. Simon Sivertsen',
       'Ware, Mr. Frederick', 'Peter, Master. Michael J'], dtype=object)

In [0]:
import re
listy = []
for name in traintest["Name"]:
  s = re.escape(", ")
  e = re.escape(". ")
  r = re.search('%s(.*)%s' % (s, e), name).group(1)
  listy.append(r)

traintest["Name"] = listy

In [0]:
traintest["Name"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Mrs. Martin (Elizabeth L', 'Lady', 'Sir', 'Mlle', 'Col',
       'Capt', 'the Countess', 'Jonkheer', 'Dona'], dtype=object)

In [0]:
mister = ["Don", "Rev", "Dr", "Major", "Col", "Capt", "Jonkheer", "Sir"]
mistress = ["Mme", "Ms", "Lady", "Mlle", "Dona", "the Countess", "Mrs. Martin (Elizabeth L"]

for _ in range(len(listy)):
  if listy[_] in mister:
    traintest["Name"].values[_] = "Mr"
  else:
    if listy[_] in mistress:
      traintest["Name"].values[_] = "Mrs"

In [0]:
traintest["Name"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master'], dtype=object)

In [0]:
temp = pd.factorize(traintest["Name"])
traintest["Name"] = temp[0]

In [0]:
traintest.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,3,0,1,22.0,1,0,0
1,1,1,0,38.0,1,0,1
2,3,2,0,26.0,0,0,0
3,1,1,0,35.0,1,0,0
4,3,0,1,35.0,0,0,0


In [0]:
temp = traintest.groupby("Name")["Age"].mean()
traintest["Age"].fillna(value = traintest["Name"].map(temp), inplace = True)

In [0]:
"""
eldest = traintest["Age"].max()
youngest = traintest["Age"].min()
avg = traintest["Age"].mean()

traintest["Age"] = (traintest["Age"] - avg)/(eldest - youngest)
"""

'\neldest = traintest["Age"].max()\nyoungest = traintest["Age"].min()\navg = traintest["Age"].mean()\n\ntraintest["Age"] = (traintest["Age"] - avg)/(eldest - youngest)\n'

In [0]:
traintest["Family"]  = 0
for _ in range(len(traintest)):
  traintest["Family"].values[_] = traintest["Parch"].values[_] + traintest["SibSp"].values[_]
  if traintest["Family"].values[_] > 0:
    traintest["Family"].values[_] = 1

In [0]:
traintest.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,Family
0,3,0,1,22.0,1,0,0,1
1,1,1,0,38.0,1,0,1,1
2,3,2,0,26.0,0,0,0,0
3,1,1,0,35.0,1,0,0,1
4,3,0,1,35.0,0,0,0,0


In [0]:
xtrain = traintest[:801]
xcv = traintest[801:891]
xtest = traintest[891:]

ytrain = labels[:801]
ycv = labels[801:]

In [0]:
#from sklearn import tree,linear_model,neural_network
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

In [0]:
#decisiontree
clf = DecisionTreeClassifier(criterion = "entropy")
clf.fit(xtrain, ytrain)
clf.score(xcv, ycv)

0.8

In [0]:
#MultiLayerPerceptron
clf1 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(18), random_state=0)
clf1.fit(xtrain, ytrain)
clf1.score(xcv, ycv)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8666666666666667

In [0]:
#LogisticReg
clf2 = LogisticRegression(C = 1000, random_state = 1)
clf2.fit(xtrain, ytrain)
clf2.score(xcv, ycv)

0.8555555555555555

In [0]:
#AdaBoost
clf3 = AdaBoostClassifier(n_estimators = 100)
clf3.fit(xtrain, ytrain)
clf3.score(xcv, ycv)

0.8333333333333334

In [0]:
#RandomForest
clf4 = RandomForestClassifier(n_estimators = 100, max_features = 7)
clf4.fit(xtrain, ytrain)
clf4.score(xcv, ycv)

0.8444444444444444

In [0]:
#BaggingClassifier
clf5 = BaggingClassifier(base_estimator = clf1, n_estimators = 100, random_state = 7)
clf5.fit(xtrain, ytrain)
clf5.score(xcv, ycv)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

0.6888888888888889

In [0]:
y = clf.predict(xtrain)
y1 = clf1.predict(xtrain)
y2 = clf2.predict(xtrain)
y3 = clf3.predict(xtrain)
y4 = clf4.predict(xtrain)
y5 = clf5.predict(xtrain)

Xboost = np.vstack([y, y1, y2, y3, y4, y5]).T

In [0]:
clf6 = GradientBoostingClassifier()
clf6.fit(Xboost, ytrain)
clf6.score(Xboost, ytrain)

0.9388264669163545

In [0]:
yp = clf.predict(xtest)
yp1 = clf1.predict(xtest)
yp2 = clf2.predict(xtest)
yp3 = clf3.predict(xtest)
yp4 = clf4.predict(xtest)
yp5 = clf5.predict(xtest)

Yboost = np.vstack([yp, yp1, yp2, yp3, yp4, yp5]).T

In [0]:
predictions = clf6.predict(Yboost)

In [0]:
predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [0]:
pred = pd.read_csv("gender_submission.csv")
pred["Survived"] = predictions

In [0]:
pred.to_csv("pred_!age_norm_family.csv", index = False)

KNN approach

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7, weights = "distance")
knn.fit(xtrain, ytrain)
knn.score(xcv, ycv)

0.8111111111111111

In [0]:
predKNN = knn.predict(xtest)
pred["Survived"] = predKNN

In [0]:
pred.to_csv("knn.csv", index = False)